<a href="https://colab.research.google.com/github/Abhiramiraju003/Multilayerperception-/blob/main/Multilayer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy  scikit-learn tensorflow transformers fasttext matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313471 sha256=12893de3dd1fe58aa80b5994a41da98c2e1a5dc05801a446711f283991460064
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import fasttext


In [ ]:
df=pd.read_csv('/content/1429_1.csv')

<ipython-input-3-42c3cc30ba5a>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/1429_1.csv')


In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    return ' '.join(text)

In [ ]:
import nltk # Import the nltk library
nltk.download('stopwords')
nltk.download('punkt_tab')  # Download the punkt_tab data package
stop_words = set(nltk.corpus.stopwords.words('english'))

def preprocess_text(text):
    # Check if the text is a string before applying lowercasing
    if isinstance(text, str):
        text = text.lower()
        text = nltk.word_tokenize(text)
        text = [word for word in text if word not in stop_words]
        return ' '.join(text)
    else:
        # Handle non-string values (e.g., NaN)
        return ""  # Or any other appropriate handling
# Print the columns of your DataFrame to see the actual column names
print(df.columns)

# Assuming the actual column name containing text data is 'review_text'
df['reviews.text'] = df['reviews.text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['reviews.text'], df['reviews.rating'], test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# Train a Multilayer Perceptron (MLP) classifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score # Import necessary functions
#Import compute_class_weight
from sklearn.utils.class_weight import compute_class_weight
# Import MLPClassifier
from sklearn.neural_network import MLPClassifier
X = df['reviews.text']
y = df['reviews.rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform both the training and testing data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Compute class weights
y_train = y_train.fillna(0).astype(int)
y_test = y_test.fillna(0).astype(int)
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train) # Call the imported function




# Train a Multilayer Perceptron (MLP) classifier with class weighting
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000)
mlp.fit(X_train_tfidf, y_train)
# Evaluate the MLP model
y_pred_mlp = mlp.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_mlp)
print("MLP Accuracy:", accuracy)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred_mlp, average='weighted')  # Change here
recall = recall_score(y_test, y_pred_mlp, average='weighted')  # Change here
f1 = f1_score(y_test, y_pred_mlp, average='weighted')  # Change here
print("MLP Precision:", precision)
print("MLP Recall:", recall)
print("MLP F1 Score:", f1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_mlp)
print("MLP Confusion Matrix:\n", conf_matrix)

# Calculate additional metrics from the confusion matrix
tp = conf_matrix[1, 1]  # True Positives
tn = conf_matrix[0, 0]  # True Negatives
fp = conf_matrix[0, 1]  # False Positives
fn = conf_matrix[1, 0]  # False Negatives

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)
print("MLP Specificity:", specificity)

# False Positive Rate (1 - Specificity)
fpr = fp / (fp + tn)
print("MLP False Positive Rate:", fpr)

# False Negative Rate
fnr = fn / (fn + tp)
print("MLP False Negative Rate:", fnr)

# Get user input
user_input = input("Please enter a review or sentence: ")

# Convert user input to TF-IDF vector
user_input_tfidf = vectorizer.transform([user_input])

# Predict sentiment using the trained MLP model
y_pred_user = mlp.predict(user_input_tfidf)

# Print predicted sentiment
print("Predicted Sentiment:")
print("Positive" if y_pred_user == 1 else "Negative")

MLP Accuracy: 0.6217541834968263
MLP Precision: 0.6225846060591274
MLP Recall: 0.6217541834968263
MLP F1 Score: 0.621822056300573
MLP Confusion Matrix:
 [[   2    0    0    0    1    7]
 [   0    6   15    8   17   34]
 [   0    3    3   15   29   31]
 [   0    6   12   52  101  107]
 [   0    9   25  138  581  946]
 [   1   21   34  124  938 3666]]
MLP Specificity: 1.0
MLP False Positive Rate: 0.0
MLP False Negative Rate: 0.0
Please enter a review or sentence: i hate this product
Predicted Sentiment:
Negative
